Please remember to run  **pip3 install textattack[tensorflow]**  in your notebook enviroment before the following codes:

# Explain Attacking BERT models using CAptum

Captum is a PyTorch library to explain neural networks
Here we show a minimal example using Captum to explain BERT models from TextAttack

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QData/TextAttack/blob/master/docs/2notebook/Example_5_Explain_BERT.ipynb)

[![View Source on GitHub](https://img.shields.io/badge/github-view%20source-black.svg)](https://github.com/QData/TextAttack/blob/master/docs/2notebook/Example_5_Explain_BERT.ipynb)

In [1]:
import torch
import pandas as pd
from copy import deepcopy
import time
import GPUtil

from BERTweet.TweetNormalizer import *

In [2]:
from textattack.datasets import Dataset
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.models.wrappers import ModelWrapper
from transformers import AutoModelForSequenceClassification, AutoTokenizer

from textattack import Attacker, AttackArgs

In [3]:
from captum.attr import IntegratedGradients, LayerConductance, LayerIntegratedGradients, LayerDeepLiftShap, InternalInfluence, LayerGradientXActivation
from captum.attr import visualization as viz

In [4]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")

print(device)

cuda:0


In [5]:
df = pd.read_csv("data/train.csv")
df["tweet"] = normalizeTweet(df["tweet"].values)
df = df.sample(frac=1, random_state=12345678)

In [6]:
dataset = Dataset(df[["tweet", "label"]].values)
original_model = AutoModelForSequenceClassification.from_pretrained("results_mask(1-10 are corrupted)/model_28/")
original_tokenizer = AutoTokenizer.from_pretrained("vinai/bertweet-large", use_fast=False)
model = HuggingFaceModelWrapper(original_model,original_tokenizer)

In [7]:
def get_text(tokenizer,input_ids,token_type_ids,attention_mask):
    list_of_text = []
    number = input_ids.size()[0]
    for i in range(number):
        ii = input_ids[i,].cpu().numpy()
        tt = token_type_ids[i,]
        am = attention_mask[i,]
        txt = tokenizer.decode(ii, skip_special_tokens=True)
        list_of_text.append(txt)
    return list_of_text

sel = 2
batch_encoded = model.tokenizer([dataset[i][0]['text'] for i in range(sel)], padding=True, return_tensors="pt")
batch_encoded.to(device)

labels = [dataset[i][1] for i in range(sel)]

clone = deepcopy(model)
clone.model.to(device)

def calculate(input_ids, attention_mask):
    #convert back to list of text
    return clone.model(input_ids, attention_mask=attention_mask)[0]

# x = calculate(**batch_encoded)

lig = LayerIntegratedGradients(calculate, clone.model.roberta.embeddings)
# lig = InternalInfluence(calculate, clone.model.bert.embeddings)
# lig = LayerGradientXActivation(calculate, clone.model.bert.embeddings)

bsl = torch.zeros(batch_encoded['input_ids'].size()).type(torch.LongTensor).to(device)
labels = torch.tensor(labels).to(device)

attributions,delta = lig.attribute(inputs=batch_encoded['input_ids'],
                              baselines=bsl,
                              additional_forward_args=(batch_encoded['attention_mask']),
                              n_steps = 10,
                              target = labels,
                              return_convergence_delta=True
                              )
atts = attributions.sum(dim=-1).squeeze(0)
atts = atts / torch.norm(atts)

In [8]:
atts = attributions.sum(dim=-1).squeeze(0)
atts = atts / torch.norm(atts)

In [9]:
from textattack.attack_recipes import PWWSRen2019
attack = PWWSRen2019.build(model)

[nltk_data] Downloading package omw-1.4 to /home/alexlu/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
textattack: Unknown if model of class <class 'transformers.models.roberta.modeling_roberta.RobertaForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


In [10]:
import pickle

results = []

# with open("textattackresults.pkl", "rb") as f:
#     results = pickle.load(f)

len(results)

0

In [11]:
count = 0
from textattack.attack_results.successful_attack_result import SuccessfulAttackResult
for i in results:
    if isinstance(i, SuccessfulAttackResult):
        count += 1

In [12]:
count

0

In [13]:
x = 0

for i in range(x, 20):
    print("="*20 + str(i) + "="*20)
    attack_args = AttackArgs(
        num_successful_examples=100,
        num_examples_offset=len(results),
        log_to_csv=f"attacks/{i}.csv",
        disable_stdout=True,
        # parallel=True,
        # num_workers_per_device=4
    )

    attacker = Attacker(attack, dataset, attack_args)
    res = attacker.attack_dataset()
    results.extend(res)
    
    with open("textattackresults.pkl", "wb") as f:
        pickle.dump(results, f)

    if GPUtil.getGPUs()[0].temperature > 70:
        while GPUtil.getGPUs()[0].temperature > 60:
            time.sleep(1)
    

textattack: Logging to CSV at path attacks/0.csv


====================0====================
Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapWordNet
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 100 / 89 / 0 / 189: 100%|██████████| 100/100 [26:42<00:00, 16.03s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 100    |
| Number of failed attacks:     | 89     |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 47.09% |
| Attack success rate:          | 52.91% |
| Average perturbed word %:     | 14.5%  |
| Average num. words per input: | 38.87  |
| Avg num queries:              | 339.53 |
+-------------------------------+--------+


textattack: Logging to CSV at path attacks/1.csv


...


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 100    |
| Number of failed attacks:     | 112    |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 52.83% |
| Attack success rate:          | 47.17% |
| Average perturbed word %:     | 15.04% |
| Average num. words per input: | 38.82  |
| Avg num queries:              | 349.92 |
+-------------------------------+--------+


textattack: Logging to CSV at path attacks/19.csv



====================19====================
Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapWordNet
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 100 / 133 / 0 / 233: 100%|██████████| 100/100 [38:45<00:00, 23.26s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 100    |
| Number of failed attacks:     | 133    |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 57.08% |
| Attack success rate:          | 42.92% |
| Average perturbed word %:     | 15.38% |
| Average num. words per input: | 39.87  |
| Avg num queries:              | 367.18 |
+-------------------------------+--------+


In [14]:
# from IPython.core.display import HTML

# df = pd.read_csv("attacks.csv")
# display(HTML(df.to_html(escape=False)))